In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
paths = glob.glob('../input/torch*')

In [ ]:
!cp -r ../input/youdontknow/submission_median_round_LB153.csv ./

In [ ]:
class config:
    paths = {
        # data
        'train': '../input/ventilator-pressure-prediction/train.csv',
        'test' : '../input/ventilator-pressure-prediction/test.csv',
        'ss'   : '../input/ventilator-pressure-prediction/sample_submission.csv',    
    }
    
    model_params = {
        'is_train': True,
        'debug'   : False,
        
        'EPOCH'   : 300,
        'BATCH_SIZE' : 1024,
        'NUM_FOLDS' : 10,
    }
    
    post_processing = {
        'max_pressure': 64.82099173863948,
        'min_pressure': -1.8957442945646408,
        'diff_pressure': 0.07030215,
    }

In [ ]:
df1 = pd.read_csv('../input/keraslstm151/submission_median_round_LB153.csv')
df2 = pd.read_csv('../input/lstmfold10146/submission_median_round_LB153.csv')
df1.pressure = df1.pressure*0.15 + df2.pressure*0.85
submission = df1
submission["pressure"] = np.round((submission.pressure - config.post_processing["min_pressure"])/config.post_processing["diff_pressure"]) * config.post_processing["diff_pressure"] + config.post_processing["min_pressure"]
submission.pressure = np.clip(submission.pressure, config.post_processing["min_pressure"], config.post_processing["max_pressure"])
df1.to_csv('sub.csv',index = False)

In [ ]:
df = pd.concat([pd.read_csv(p+'/oof.csv') for p in paths])
df = df[df.pred!=0]

In [ ]:
np.mean(np.abs(df['pred']-df['pressure']))

In [ ]:
sub = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
sub['pressure'] = np.mean([pd.read_csv(p+'/submission.csv')['pressure'].values for p in paths],0)

In [ ]:
sub.to_csv('submission.csv',index = False)